# Step 2 - Characterisation (MacOS)

In this notebook, you will learn more about the second step of an SDL: Characterisation.

You will need:
1. The computer in front of you with this notebook open.
2. A BBC microbit.
3. A enviro:bit extension board. (buy [here](https://shop.pimoroni.com/products/enviro-bit)) 

### For the best experience, following along the video-series that explains the notebooks step by step [here](http://sdl4kids.com).

If you have any questions, check out the [FAQ](https://sites.google.com/matterhorn.studio/sdl4kids/faq)!

## Introduction

We will look at all the building blocks required for measuring the RGB color on your screen, that you learned how to synthesise in Step 1.

1. Setting up the microbit to measure colors
2. Programming the microbit to send measurements
3. Processing the measurements

## (1) Setting up the microbit to measure colors

**Goal**: We will setup our micro:bit with the enviro:bit (PIM355) by Pimoroni which is available for purchase [here](https://shop.pimoroni.com/products/enviro-bit?variant=3142289653770). 

#### The enviro:bit has a color sensor which we will use to measure RGB colors on our computer screen.

It is the most simple way to measure RGB colors with a microbit (or any microcontroller):
1. Plug the microbit into the board. Make sure you slide it in the right way (the side with 'envirobit' on it should be on the same side where it says 'microbit' on the microbit), like [this](https://shop.pimoroni.com/products/enviro-bit?variant=3142289653770).
2. Then plug the microbit via USB into your computer. The LEDs should light up.
3. Open MakeCode where we will program the microbit on [https://makecode.microbit.org/](https://makecode.microbit.org/). MakeCode works best in [Google Chrome](https://www.google.com/chrome/)
4. Click "New Project" and call it "SDL4Kids". You should now have the programming interface in front of you.

**Task**: If you are unfamiliar with the microbit, make sure to try some of the tutorials and smaller programming projects to make sure you understand the workflow.

## (2) Programming the Microbit to send new RGB color measurements

**Goal**: We will program the microbit to send us RGB values.

Closely follow these steps:

1. In the middle column, somewhere between "Math" and "Variables" click on "Extensions".
2. Search for "envirobit", then click on it to load the envirobit library into your Microbit program.
3. User drag and drop to create below program into your Microbit. 
4. Make sure you include the dash "-" (or minus sign) between the measurements within the orange 'join' block.

*Hint*: The 'serial' blocks are hidden in the 'Advanced' section. 
The orange 'join' block is in the 'Text' section. The red music block is in the 'Music' section.

![Microbit Program](img/microbit_program.png "Microbit Program")

5. Alternatively, you can copy and paste the following code into the "JavaScript" section (switch to JavaScript via its button in the middle of the top bar, NOT Python):
```
serial.onDataReceived(serial.delimiters(Delimiters.Comma), function () {
    serial.writeLine("" + envirobit.getRed() + "-" + envirobit.getGreen() + "-" + envirobit.getBlue())
    music.playTone(262, music.beat(BeatFraction.Sixteenth))
})
music.setVolume(127)

```

6. Connect to your microbit by clicking 'Connect' on the bottom left. Follow the instructions on the screen.
7. Once connected, press download to load the program onto the microbit.
8. **Alternatively**, follow the standard programming procedure by downloading the program and copying onto the microbit, see [here](https://microbit.org/get-started/first-steps/set-up/#transfer-from-a-computer).



## (3) Connecting to the Microbit and requesting new measurements

**Goal**: Connect to the microbit and request it to send some color measurements.

**Note**: Ideally, you disconnect your Microbit from Google Chrome before continuing. Otherwise, Google Chrome will interfere with the serial connection and a lot of measurements will be corrupted.

Let's import the 'pyserial' package, which we use to connect to the Microbit (it is just called 'serial'):

In [ ]:
import serial
import numpy as np

We will list all USB devices currently connected to our computer:

In [ ]:
!ls /dev/cu.*

Choose the one with 'cu.usbmodem' in its name and copy its FULL name into the code below:

In [ ]:
ser = serial.Serial()
name = "/dev/cu.usbmodem102"
ser = serial.Serial(name,115200, timeout=0.1)

The following lines of code will send a comma (",") to the microbit, which is our signal to it to send us the RGB measurements.

In [ ]:
# 1. Send a comma to the microbit. The b in b"," indicates that it should be in bytecode
ser.write(b",")
# 2. We then read the response from the microbit. 
serial_data = str(ser.readline().decode('utf8')).rstrip()
serial_data

The first time will probably fail. Run it a few more times. You should now see three numbers separated by a dash, something like '115-92-103'.

**Task**: Move around the microbit and request new messages. Are the numbers changing? What colors can you measure?

For the next steps, make sure you put the envirobit sensor on your screen like this: ![Measuring colors](img/measuring_colors.png "Measuring colors")

## (4) Making the connection robust

**Goal**: Add validation procedure to reduce noise.

Unfortunately, depending on your computer system, the serial USB connection between the microbit and computer might not be robust and messages will get corrupted.

To avoid these messages breaking our SDL, we will add a validation procedure:
1. If there are not exactly three numbers, we will measure again.
2. If the numbers are not between 0 and 255, we will measure again.
3. If validation fails 25 times in a row, we give up, as the issue must lay somewhere else, requiring deeper debugging.

This procedure is not perfect, as the numbers themselves could be corrupted, but we will just have to accept that as measurement noise at this stage.

We will introduce a function called 'characterise' that returns the measured RBG color.

In [ ]:
def characterise():
    import time
    valid = False
    max_num_tries = 25
    num_tries = 0
    while not valid:
        try:
            ser.flushInput()
            ser.write(b",")
            serial_data = str(ser.readline().decode('utf8')).rstrip()
            rgb = tuple([int(value) for value in serial_data.split("-")])
            # Measurement needs three entries
            if len(rgb) != 3:
                print(f"Measurement did not containt 3 RGB values:{rgb}")
                raise Exception()
            if not np.all([0 <= x <= 255 for x in rgb]):
                print(f"Measurement outside valid range [0,255]:{rgb}")
                raise Exception()
            valid = True
        except Exception as err:
            print(f"Unexpected {err=}, {type(err)=}")
            print("Measurement invalid, will try again")
            if num_tries > max_num_tries:
                print("Please check connection. Maximum number of tries exceeded. There seems to be a fatal error.")
                break
            num_tries = num_tries + 1
    print(f"New RGB Measurement: {rgb}")
    time.sleep(0.4)
    return rgb

Let's give it a try:

In [ ]:
characterise()

Let's take 50 measurements:

In [ ]:
for i in range(50):
    characterise()

If you do not move the microbit much, you will not see the numbers change much.

**Task**: Play around with it. Move the microbit a bit around. Do the numbers change?

## (5) Putting it all together

We can now measure colors with the microbit using a function called 'characterise()'.

Let's take add this function into the code from the loop we programmed in Step 1:

1. We will still generate random colors.
2. Instead of calculating the error with a color we generated in the computer, we will measure the color with the microbit and measure its error to the goal color.

The change is very small, just one line. Can you find it?

In [ ]:
import cv2
import numpy as np

import random #this package give us access to a function called 'randint(lowest, highest)'
def generate_random_color():
    red = random.randint(0, 255)
    green = random.randint(0, 255)
    blue = random.randint(0, 255)
    return blue, green, red  # OpenCV uses BGR color format

import numpy as np # We import numpy so we can use it's square root function np.sqrt()
def calculate_error(color1, color2):
    return np.sqrt((color1[0] - color2[0]) ** 2 + (color1[1] - color2[1]) ** 2 + (color1[2] - color2[2]) ** 2)

# 1. Create a named window for display
cv2.namedWindow("Synthesis Window", cv2.WINDOW_NORMAL)
cv2.startWindowThread()

# 2. Create a blank image
width, height = 800, 400
image = np.zeros((height, width, 3), dtype=np.uint8)

# 3.1 Define how many colors we want to show
max_iterations = 10

# 3.2 Let's define our goal color
goal_color = (0,0,255) # BGR order, not RGB

# 3.3 Create an empty list to store the error values
error_values = []

# 3.4 Define font properties for displaying text
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.8
font_color = (255, 255, 255)  # White color

for iteration in range(max_iterations):
    # 4. Generate a new random color
    current_color = generate_random_color()
    
    # 5.1 Update the live image with the current color
    image[:, :] = tuple(reversed(current_color))
    
    # 5.2 Let's plot the goal color
    subplot_x = width // (max_iterations + 1)
    subplot_width = width // (max_iterations + 1)
    cv2.rectangle(image, (subplot_x, 60), (subplot_x + subplot_width, height - 60), goal_color, -1)
    
    # 5.3 Let's also calculate the difference between the MEASURED color and the goal
    current_color = characterise() # THIS IS THE CHANGE!
        
    error = calculate_error(goal_color, current_color)
    error_values.append(error)
    print(error)
    
    # 5.4. Add text information to the image
    text = f"Iteration: {iteration + 1}"
    cv2.putText(image, text, (10, 30), font, font_scale, font_color, 2)

    # 5.5 Add text for the error
    error_text = f"Loss: {error:.2f}"
    cv2.putText(image, error_text, (10, height - 10), font, font_scale, font_color, 2)

    # 5.6 Draw the graph of error values
    if len(error_values) > 1:
        for i in range(1, len(error_values)):
            x1 = (i - 1) * (width // max_iterations)
            y1 = height - int(error_values[i - 1] * (height - 60) / max(error_values))
            x2 = i * (width // max_iterations)
            y2 = height - int(error_values[i] * (height - 60) / max(error_values))
            cv2.line(image, (x1, y1), (x2, y2), (255, 255, 255), 2)
    
    # 6. Display the image again
    cv2.imshow("Live Image", image)
    cv2.waitKey(1) 
        
# Close the window after the desired number of iterations
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

Let's plot the error again:

In [ ]:
import matplotlib.pyplot as plt
# Plot error for each iteration
plt.scatter(range(len(error_values)), error_values, label="Error for each iteration")

# Calculate best error so far
best_error_so_far = 1000
best_errors = []
for value in error_values:
    if value < best_error_so_far:
        best_error_so_far = value
    best_errors.append(best_error_so_far)
    
# Plot best error so far
plt.plot(best_errors, label="Best Error So Far", color="orange")
plt.xlabel('Iteration', fontsize=18)
plt.ylabel('Distance to Goal', fontsize=18)
plt.legend()

**Question**: Which RGB combination was closest to our goal color?

**Solution**: Check the loss graph for the combination with the smallest loss, i.e. it is closest to our goal color.

In [ ]:
iteration_with_smallest_loss = error_values.index(min(error_values))
print("The smallest loss is " + str(error_values[iteration_with_smallest_loss]) + " with the iteration number of " + str(iteration_with_smallest_loss))

**Task**: Increase the number of iterations. Do the colors and their calcualted errors match with what you see?

### **Now it's time to move on to the third and final step of an SDL: Planning. Find it in the notebook for Step 3**